<a href="https://colab.research.google.com/github/kazzastic/BreastCancer-Deep-Learning/blob/master/Patches_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.regularizers import l2

In [0]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape = (227,227,3), pooling = 'avg')
x = base_model.output
preds = Dense(3, activation='softmax', kernel_regularizer=l2(0.01))(x)
model = Model(inputs=base_model.input, outputs=preds)
top_layer_nb = 15

In [0]:
org_model = model

In [0]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
# Model saving callback

checkpointer = ModelCheckpoint(filepath='Patch_CNN.h5', monitor='val_acc', verbose=1, save_best_only=True)

# Early stopping
early_stopping = EarlyStopping(monitor='val_acc', verbose=1, patience=5)

#ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.5, partience = 2, verbose = 1)

# First Stage Of training with learning rate 0.001

In [0]:
pos_cls_weight, neg_cls_weight = 1., 1.
for layer in org_model.layers[:-1]:
  layer.trainable = False
class_weight = { 0:1.0, 1:pos_cls_weight, 2:neg_cls_weight }
nb_worker = 1
if nb_worker == 1:
  pickle_safe = False
else:
  pickle_safe = True


In [0]:
train_data_dir = "/content/drive/My Drive/patches_sort"

In [56]:
datagen = ImageDataGenerator(validation_split=0.3)
batch_size = 32
img_width , img_height = 227, 227

# Retrieve images and their classes for train and validation sets
train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        shuffle = True,
        batch_size=batch_size,
        #class_mode='binary',
        subset = 'training')

validation_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        shuffle = True,
        batch_size=batch_size,
        #class_mode='binary',
        subset = 'validation')

Found 40530 images belonging to 3 classes.
Found 17367 images belonging to 3 classes.


In [57]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'],
              )
model.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 227, 227, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 227, 227, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 227, 227, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 113, 113, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 113, 113, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 113, 113, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

Run the cell below for about 3 to 4 epoch

In [0]:
hist = model.fit_generator(
        train_generator,
        steps_per_epoch=32,
        epochs=100,
        validation_data = validation_generator,
        validation_steps = 32,
        callbacks=[checkpointer, early_stopping],
        verbose=1)
        #class_weight = class_weight)

Epoch 1/100
32/32 [==============================] - 229s 7s/step - loss: 1.4545 - acc: 0.3633

Epoch 00001: val_acc improved from -inf to 0.36328, saving model to Patch_CNN.h5
32/32 [==============================] - 428s 13s/step - loss: 1.6418 - acc: 0.3652 - val_loss: 1.4545 - val_acc: 0.3633
Epoch 2/100
32/32 [==============================] - 4s 127ms/step - loss: 1.2980 - acc: 0.3955

Epoch 00002: val_acc improved from 0.36328 to 0.39551, saving model to Patch_CNN.h5
32/32 [==============================] - 136s 4s/step - loss: 1.3935 - acc: 0.3604 - val_loss: 1.2980 - val_acc: 0.3955
Epoch 3/100
32/32 [==============================] - 4s 130ms/step - loss: 1.2162 - acc: 0.4209

Epoch 00003: val_acc improved from 0.39551 to 0.42090, saving model to Patch_CNN.h5
32/32 [==============================] - 194s 6s/step - loss: 1.2883 - acc: 0.3613 - val_loss: 1.2162 - val_acc: 0.4209
Epoch 4/100
32/32 [==============================] - 4s 128ms/step - loss: 1.1995 - acc: 0.4160

Epo